# Models

The current models being implemented are namely:

* Supervised
    1. Logistic Regression Model
    2. Random Forest Classifier
    3. Support Vector Machine
    4. Deep Neural Network
    5. (New) Bayesian Network
    6. (New) Gradient Boosting

* Unsupervised
    1. (New) K-Means Clustering
    2. (New) Kernel Density Estimation

In [ ]:
import importlib

### Logistic Regression

In [ ]:
import logisticRegression
importlib.reload(logisticRegression)

LR = logisticRegression.LogisticRegressionModel("CSV/wave1_select_labelled.csv")

LR.run()

Validation Accuracy: 1.00
Validation Confusion Matrix:
[[2020    2]
 [   4 1043]]
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2022
           1       1.00      1.00      1.00      1047

    accuracy                           1.00      3069
   macro avg       1.00      1.00      1.00      3069
weighted avg       1.00      1.00      1.00      3069

Test Accuracy: 1.00
Test Confusion Matrix:
[[2030    2]
 [   5 1032]]
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2032
           1       1.00      1.00      1.00      1037

    accuracy                           1.00      3069
   macro avg       1.00      1.00      1.00      3069
weighted avg       1.00      1.00      1.00      3069

Best Parameters: {'logisticregression__C': 0.1, 'logisticregression__penalty': 'l1'}
Best Score: 1.0


### Random Forest

In [ ]:
import randomForest
importlib.reload(randomForest)

RF = randomForest.RandomForestModel("CSV/wave1_select_labelled.csv")

RF.run()

Validation Accuracy: 1.00
Validation Confusion Matrix:
[[2022    0]
 [   0 1047]]
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2022
           1       1.00      1.00      1.00      1047

    accuracy                           1.00      3069
   macro avg       1.00      1.00      1.00      3069
weighted avg       1.00      1.00      1.00      3069

Test Accuracy: 1.00
Test Confusion Matrix:
[[2032    0]
 [   0 1037]]
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2032
           1       1.00      1.00      1.00      1037

    accuracy                           1.00      3069
   macro avg       1.00      1.00      1.00      3069
weighted avg       1.00      1.00      1.00      3069



Testing: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}:  10%|█         | 11/108 [00:38<05:37,  3.48s/it]


KeyboardInterrupt: 

### Support Vector Machine

In [ ]:
import supportVectorMachine
importlib.reload(supportVectorMachine)

SVM = supportVectorMachine.SVMModel("CSV/wave1_select_labelled.csv")

SVM.run()

Validation Accuracy: 0.66
Validation Confusion Matrix:
[[2022    0]
 [1047    0]]
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.66      1.00      0.79      2022
           1       0.00      0.00      0.00      1047

    accuracy                           0.66      3069
   macro avg       0.33      0.50      0.40      3069
weighted avg       0.43      0.66      0.52      3069

Test Accuracy: 0.66
Test Confusion Matrix:
[[2032    0]
 [1037    0]]
Test Classification Report:
              precision    recall  f1-score   support

           0       0.66      1.00      0.80      2032
           1       0.00      0.00      0.00      1037

    accuracy                           0.66      3069
   macro avg       0.33      0.50      0.40      3069
weighted avg       0.44      0.66      0.53      3069



KeyboardInterrupt: 